In [6]:
!pip install -U langchain langchain-community langchain-core chromadb pypdf langchain-openai

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import os


In [ ]:
os.environ["OPENAI_API_KEY"] = "PASTE_YOUR_REAL_KEY_HERE"

print("Key loaded:", os.environ.get("OPENAI_API_KEY") is not None)


Key loaded: True


In [7]:
import os
print(os.listdir("/content"))

from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [8]:
!ls /content

aws_rag_guide.pdf  sample_data


In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/aws_rag_guide.pdf")
documents = loader.load()

print("Pages loaded:", len(documents))


Pages loaded: 82


In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = splitter.split_documents(documents)
print("Chunks created:", len(chunks))


Chunks created: 238


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

db = Chroma.from_documents(chunks, embedding=embeddings)
retriever = db.as_retriever(search_kwargs={"k": 5})

print("✅ Vector store created (FREE embeddings)")


/tmp/ipython-input-2769539941.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store created (FREE embeddings)


In [12]:
llm = ChatOpenAI(temperature=0)


In [13]:
prompt = ChatPromptTemplate.from_template(
    """Answer the question using only the context below.
If not found, say: "Not available in document."

Context:
{context}

Question:
{question}
"""
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

print("✅ Retrieval chain created")


✅ Retrieval chain created


In [14]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

hf_pipeline = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)
print("✅ Free LLM loaded")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FlexOlmoF

✅ Free LLM loaded


/tmp/ipython-input-2666504667.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [15]:
prompt = ChatPromptTemplate.from_template(
    """Answer the question using only the context below.
If not found, say: "Not available in document."

Context:
{context}

Question:
{question}
"""
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)


In [16]:
query = "What are fully managed RAG options in AWS?"
response = chain.invoke(query)


Token indices sequence length is longer than the specified maximum sequence length for this model (2405 > 512). Running this sequence through the model will result in indexing errors


In [17]:
print(response)


Human: Answer the question using only the context below.
If not found, say: "Not available in document."

Context:
[Document(metadata={'total_pages': 82, 'page_label': '11', 'producer': 'Apache FOP Version 2.6', 'page': 14, 'creationdate': '2025-12-23T00:02:33+00:00', 'creator': 'ZonBook XSL Stylesheets with Apache FOP', 'keywords': 'AWS Prescriptive Guidance, Retrieval Augmented Generation, generative ai, custom documents, artificial intelligence and machine learning, large language model (LLM), Amazon Bedrock, Amazon Kendra, Amazon SageMaker AI, Amazon OpenSearch Service, Amazon Q Business, Amazon MemoryDB, Amazon DocumentDB (with MongoDB compatibility), AWS Prescriptive Guidance, glossary, definitions', 'author': 'AWS', 'source': '/content/aws_rag_guide.pdf', 'title': 'AWS Prescriptive Guidance - Retrieval Augmented Generation options and architectures on AWS'}, page_content='AWS Prescriptive Guidance Retrieval Augmented Generation options and architectures on AWS\nFully managed Ret

In [18]:
queries = [
    "What is Retrieval Augmented Generation?",
    "Which AWS services support RAG?",
    "What are the benefits of RAG?",
    "How does AWS implement RAG?"
]

for q in queries:
    print("\nQUESTION:", q)
    print(chain.invoke(q))



QUESTION: What is Retrieval Augmented Generation?
Human: Answer the question using only the context below.
If not found, say: "Not available in document."

Context:
[Document(metadata={'page_label': '4', 'source': '/content/aws_rag_guide.pdf', 'total_pages': 82, 'producer': 'Apache FOP Version 2.6', 'creator': 'ZonBook XSL Stylesheets with Apache FOP', 'page': 7, 'keywords': 'AWS Prescriptive Guidance, Retrieval Augmented Generation, generative ai, custom documents, artificial intelligence and machine learning, large language model (LLM), Amazon Bedrock, Amazon Kendra, Amazon SageMaker AI, Amazon OpenSearch Service, Amazon Q Business, Amazon MemoryDB, Amazon DocumentDB (with MongoDB compatibility), AWS Prescriptive Guidance, glossary, definitions', 'title': 'AWS Prescriptive Guidance - Retrieval Augmented Generation options and architectures on AWS', 'creationdate': '2025-12-23T00:02:33+00:00', 'author': 'AWS'}, page_content="• Comparing Retrieval Augmented Generation and ﬁne-tuning\n

In [19]:
docs = retriever.invoke(query)

for i, doc in enumerate(docs):
    print(f"\nSource {i+1}:")
    print(doc.metadata)



Source 1:
{'producer': 'Apache FOP Version 2.6', 'author': 'AWS', 'total_pages': 82, 'creator': 'ZonBook XSL Stylesheets with Apache FOP', 'creationdate': '2025-12-23T00:02:33+00:00', 'page': 14, 'title': 'AWS Prescriptive Guidance - Retrieval Augmented Generation options and architectures on AWS', 'source': '/content/aws_rag_guide.pdf', 'page_label': '11', 'keywords': 'AWS Prescriptive Guidance, Retrieval Augmented Generation, generative ai, custom documents, artificial intelligence and machine learning, large language model (LLM), Amazon Bedrock, Amazon Kendra, Amazon SageMaker AI, Amazon OpenSearch Service, Amazon Q Business, Amazon MemoryDB, Amazon DocumentDB (with MongoDB compatibility), AWS Prescriptive Guidance, glossary, definitions'}

Source 2:
{'source': '/content/aws_rag_guide.pdf', 'total_pages': 82, 'author': 'AWS', 'producer': 'Apache FOP Version 2.6', 'title': 'AWS Prescriptive Guidance - Retrieval Augmented Generation options and architectures on AWS', 'page_label': '7

In [20]:
!pip install -q transformers accelerate torch


In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"   # lightweight & free

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256
)


Device set to use cpu


In [22]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def rag_answer(query):
    docs = retriever.invoke(query)
    context = format_docs(docs)

    prompt = f"""
    Answer the question using ONLY the context below.
    If the answer is not in the context, say "I don't know".

    Context:
    {context}

    Question:
    {query}
    """

    response = llm_pipeline(prompt)
    return response[0]["generated_text"]


In [23]:
query = "What are fully managed RAG options in AWS?"
answer = rag_answer(query)

print("Answer:\n", answer)


Token indices sequence length is longer than the specified maximum sequence length for this model (853 > 512). Running this sequence through the model will result in indexing errors


Answer:
 To manage Retrieval Augmented Generation workflows on AWS, you can use custom RAG pipelines or use some of the fully managed services capabilities that AWS offers. Because they include many of the core components of a RAG-based system, fully managed services can help you manage some of the undifferentiated heavy lifting. However, these services provide less opportunity for customization. The fully managed AWS services use connectors to ingest data from external data sources, such as websites, Atlassian Confluence, or Microsoft SharePoint. The supported data sources vary by AWS service. AWS Identity and Access Management (IAM). Clearly, there is significant amount of work to plan, develop, release, and manage a RAG system. Fully managed services, such as Amazon Bedrock or Amazon Q Business, can help you manage some of the undifferentiated heavy lifting. However, these services provide less opportunity for customization. The fully managed AWS services use connectors to ingest da